<a href="https://colab.research.google.com/github/satishgc227/Derby-project/blob/main/Accurate_deep_learning_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score




In [2]:
# Load in the data
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('/content/edited complete.csv',encoding='latin',nrows=100000)

df['race_type\xa0'].unique()
df['course_type'].unique()
df['track_condition\xa0'].unique()
df['track_id'].unique()

s=df[['race_type\xa0','course_type','track_condition\xa0','track_id']]

ohe=OneHotEncoder()
feature_array=ohe.fit_transform(df[['race_type\xa0','course_type','track_condition\xa0','track_id']]).toarray()

feature_labels=ohe.categories_
print(feature_labels)

np.hstack(feature_labels)

feature_labels=np.hstack(feature_labels)
f=pd.DataFrame(feature_array,columns=feature_labels)
pd.concat([df,f],axis=1)

df.drop(['race_date','track_id','course_type','track_condition\xa0','race_type\xa0','jockey\xa0','program_number\xa0','latitude\xa0','longitude\xa0'],axis=1,inplace=True)
df=pd.concat([df,f],axis=1)

df.isnull().any()



"""Removing the null values"""

df.replace([np.inf,-np.inf],np.nan ,inplace=True)
df.dropna(inplace=True)
df.isnull().any()

!pip install movecolumn
import movecolumn as mc
mc.MoveToLast(df,'position_at_finish\xa0')

X = df.iloc[:,:-1]
y = df.iloc[:, -1]




/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[array(['ALW', 'AOC', 'CLM', 'SOC', 'SST', 'STK', 'STR', 'WCL'],
      dtype=object), array(['D', 'I', 'M', 'O', 'T'], dtype=object), array(['FM ', 'FT ', 'GD ', 'MY ', 'SY ', 'YL '], dtype=object), array(['AQU', 'BEL', 'SAR'], dtype=object)]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [4]:
df.corr()['position_at_finish\xa0']

race_number            0.118034
trakus_index          -0.069247
distance_id           -0.085558
run_up_distance        0.102967
purse                  0.045772
post_time              0.142631
weight_carried        -0.075192
odds                   0.459306
ALW                    0.100923
AOC                   -0.052711
CLM                    0.006919
SOC                    0.029892
SST                    0.040741
STK                   -0.072638
STR                    0.079650
WCL                   -0.039686
D                     -0.093738
I                     -0.010449
M                     -0.061592
O                      0.127725
T                      0.113023
FM                     0.031105
FT                    -0.050933
GD                     0.155851
MY                     0.028887
SY                    -0.127070
YL                    -0.061817
AQU                    0.035534
BEL                   -0.026824
SAR                   -0.013751
position_at_finish     1.000000
Name: po

In [5]:
X

,race_number,trakus_index,distance_id,run_up_distance,purse,post_time,weight_carried,odds,ALW,AOC,...,T,FM,FT,GD,MY,SY,YL,AQU,BEL,SAR
0,9,73,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,9,74,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,9,63,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,9,64,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,9,65,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3,83,600,48,36000,108,123,430,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99996,3,84,600,48,36000,108,123,430,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99997,3,73,600,48,36000,108,123,430,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99998,3,74,600,48,36000,108,123,430,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [6]:
model=Sequential()
model.add(Dense(32,activation='elu', input_dim=30))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=335,epochs=44,validation_data=(X_test,y_test))

Epoch 1/44
239/239 [==============================] - 2s 4ms/step - loss: -151314.3906 - accuracy: 0.1060 - val_loss: -377664.2812 - val_accuracy: 0.0996
Epoch 2/44
239/239 [==============================] - 1s 3ms/step - loss: -828996.9375 - accuracy: 0.1060 - val_loss: -1431384.6250 - val_accuracy: 0.0996
Epoch 3/44
239/239 [==============================] - 1s 3ms/step - loss: -2212773.5000 - accuracy: 0.1060 - val_loss: -3178679.7500 - val_accuracy: 0.0996
Epoch 4/44
239/239 [==============================] - 1s 4ms/step - loss: -4244137.5000 - accuracy: 0.1060 - val_loss: -5548762.0000 - val_accuracy: 0.0996
Epoch 5/44
239/239 [==============================] - 1s 3ms/step - loss: -6866580.5000 - accuracy: 0.1060 - val_loss: -8496597.0000 - val_accuracy: 0.0996
Epoch 6/44
239/239 [==============================] - 1s 3ms/step - loss: -10030588.0000 - accuracy: 0.1060 - val_loss: -11967853.0000 - val_accuracy: 0.0996
Epoch 7/44
239/239 [==============================] - 1s 3ms/step

In [7]:
#How to select appropriate optimizers.
#!pip install -U keras-tuner

In [15]:
def model_builder(hp):
  model = keras.Sequential()
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(12))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [16]:
#!pip install -q -U keras-tuner

In [17]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras

In [18]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [19]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [20]:
tuner.search(X_train,y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [21]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train,y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
2000/2000 [==============================] - 6s 3ms/step - loss: 386.6706 - accuracy: 0.1633 - val_loss: 191.0902 - val_accuracy: 0.2412
Epoch 2/50
2000/2000 [==============================] - 7s 3ms/step - loss: 226.8868 - accuracy: 0.1937 - val_loss: 88.2952 - val_accuracy: 0.2374
Epoch 3/50
2000/2000 [==============================] - 6s 3ms/step - loss: 158.5524 - accuracy: 0.2143 - val_loss: 152.5341 - val_accuracy: 0.2319
Epoch 4/50
2000/2000 [==============================] - 6s 3ms/step - loss: 99.6730 - accuracy: 0.2359 - val_loss: 76.6495 - val_accuracy: 0.2386
Epoch 5/50
2000/2000 [==============================] - 6s 3ms/step - loss: 59.4255 - accuracy: 0.2560 - val_loss: 36.2520 - val_accuracy: 0.3016
Epoch 6/50
2000/2000 [==============================] - 6s 3ms/step - loss: 34.1435 - accuracy: 0.2746 - val_loss: 15.2424 - val_accuracy: 0.2661
Epoch 7/50
2000/2000 [==============================] - 6s 3ms/step - loss: 19.6430 - accuracy: 0.2805 - val_loss: 8.47

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train,y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/7
2000/2000 [==============================] - 7s 3ms/step - loss: 387.1011 - accuracy: 0.1656 - val_loss: 244.6476 - val_accuracy: 0.1848
Epoch 2/7
2000/2000 [==============================] - 6s 3ms/step - loss: 221.2346 - accuracy: 0.1930 - val_loss: 210.3837 - val_accuracy: 0.1841
Epoch 3/7
2000/2000 [==============================] - 6s 3ms/step - loss: 145.2760 - accuracy: 0.2139 - val_loss: 91.2465 - val_accuracy: 0.1976
Epoch 4/7
2000/2000 [==============================] - 6s 3ms/step - loss: 90.0644 - accuracy: 0.2366 - val_loss: 42.6781 - val_accuracy: 0.2986
Epoch 5/7
 924/2000 [============>.................] - ETA: 4s - loss: 64.5771 - accuracy: 0.2525